In [1]:
from time import sleep
from random import randint

In [2]:
artist = 'Weezer'
song   = 'Island in the Sun'

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
def find_link_from_search(artist, song):
    # url = 'https://www.ultimate-guitar.com/search.php?search_type=title&value=weezer%20island%20in%20the%20sun'
    # search UG for artist and song
    
    if ' - ' in song:
        song = song.split(' - ')[0]
    
    url = 'https://www.ultimate-guitar.com/search.php?search_type=title&value='+artist+'%20'+song

    # get the soup
    sleep(randint(1,4))
    text = requests.get(url).text
    soup = BeautifulSoup(text)

    # do some crafty splitting to get the tab URL
    # first few results are often ads...
    searchResults = str(soup).split('"tab_url":"')[3:6] 

    new_link = str(searchResults).split("},{")[0][2:-1]

    return new_link

In [5]:
def parse_title(url):
    artist_and_title = url.split('tab/')[1]  

    if '/' in artist_and_title:
        artist = artist_and_title.split('/')[0]  
        title  = artist_and_title.split('/')[1]
        title  = title.split('-ch')[0]
    else:
        artist = 'unknown'
        title = 'unknown'
    
    return artist, title

In [6]:
def scrape(url): 
    output_list = []
    chordlist   = []
    
    sleep(randint(1,4))
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'lxml')

    lessSoupy = html_soup.find('div', class_='js-store')
    stringySoup = str(lessSoupy)

    splitSoup = stringySoup.split('[ch]')[1:-1]

    for ielement in range(0, len(splitSoup)):
        chordlist.append(str(splitSoup[ielement]).split('[/ch')[0])
        
    artist_name, song_name = parse_title(url)
    output_list.append(("Song", song_name))
    output_list.append(("Artist", artist_name))

#     allChords = ("All Chords", chords)
    #output_list.append(allChords)

    chords = set(chordlist)
    chords = ",".join(chords)
    chords = ("Chords", chords)
    output_list.append(chords)
#     output_list.append(allChords)

    song_info = dict(output_list)

    return song_info

In [7]:
new_link = find_link_from_search(artist, song)
print(new_link)

song_info = scrape(new_link)
dfSong = pd.DataFrame.from_dict(song_info, orient='index').transpose()

dfSong

https://tabs.ultimate-guitar.com/tab/weezer/island-in-the-sun-chords-852868


,Song,Artist,Chords
0,island-in-the-sun,weezer,"Am,Em,D,C,G"


In [3]:
# get spotify credentials
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id= '6426e18767ac42d78a9c08f41cacba65',
                                                     client_secret= '4a2cdab50e1c4ecd9fc62cae20664e29')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
pl_link = 'https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=sYkoH3ZXSlKrMFtmVSp85Q'
pl_id   = pl_link.split('playlist/')[1]
pl_id   = pl_id.split('?')[0]

In [10]:
# response = sp.playlist_items(pl_id,
#                              offset=offset,
#                              fields='items.track.id,total',
#                              additional_types=['track'])

# results = spotify.user_playlist(username, playlist_id,
#                                 fields='tracks,next,name')
results = sp.user_playlist('Spotify', pl_id,
                                fields='tracks,next,name')

pl_tracks  = []
pl_artists = []

for nItem in range(0, len(results['tracks']['items'])):
    pl_tracks.append(results['tracks']['items'][nItem]['track']['name'])
    pl_artists.append(results['tracks']['items'][nItem]['track']['artists'][0]['name'])

In [12]:
pl_tab_links = []

for iTrack in range(0, len(pl_tracks)):
    tmp_link = find_link_from_search(pl_artists[iTrack], pl_tracks[iTrack])
    pl_tab_links.append(tmp_link)

In [13]:
while("" in pl_tab_links) : 
    pl_tab_links.remove("") 
    
# pl_tab_links

['https://tabs.ultimate-guitar.com/tab/jimi-hendrix/all-along-the-watchtower-chords-810679',
 'https://tabs.ultimate-guitar.com/tab/acdc/back-in-black-chords-18953',
 'https://tabs.ultimate-guitar.com/tab/david-bowie/rebel-rebel-chords-1397520',
 'https://tabs.ultimate-guitar.com/tab/the-clash/should-i-stay-or-should-i-go-chords-27335',
 'https://tabs.ultimate-guitar.com/tab/aerosmith/sweet-emotion-chords-1787420',
 'https://tabs.ultimate-guitar.com/tab/queen/another-one-bites-the-dust-chords-1090617',
 'https://tabs.ultimate-guitar.com/tab/the-doors/light-my-fire-chords-15032',
 'https://tabs.ultimate-guitar.com/tab/zz-top/la-grange-chords-2307723',
 'https://tabs.ultimate-guitar.com/tab/bob-dylan/like-a-rolling-stone-chords-138850',
 'https://tabs.ultimate-guitar.com/tab/cream/sunshine-of-your-love-chords-64026',
 'https://tabs.ultimate-guitar.com/tab/creedence-clearwater-revival/fortunate-son-chords-15889',
 'https://tabs.ultimate-guitar.com/tab/janis-joplin/me-and-bobby-mcgee-chord

In [21]:
# del pl_tab_links[15]
pl_tab_links

['https://tabs.ultimate-guitar.com/tab/jimi-hendrix/all-along-the-watchtower-chords-810679',
 'https://tabs.ultimate-guitar.com/tab/acdc/back-in-black-chords-18953',
 'https://tabs.ultimate-guitar.com/tab/david-bowie/rebel-rebel-chords-1397520',
 'https://tabs.ultimate-guitar.com/tab/the-clash/should-i-stay-or-should-i-go-chords-27335',
 'https://tabs.ultimate-guitar.com/tab/aerosmith/sweet-emotion-chords-1787420',
 'https://tabs.ultimate-guitar.com/tab/queen/another-one-bites-the-dust-chords-1090617',
 'https://tabs.ultimate-guitar.com/tab/the-doors/light-my-fire-chords-15032',
 'https://tabs.ultimate-guitar.com/tab/zz-top/la-grange-chords-2307723',
 'https://tabs.ultimate-guitar.com/tab/bob-dylan/like-a-rolling-stone-chords-138850',
 'https://tabs.ultimate-guitar.com/tab/cream/sunshine-of-your-love-chords-64026',
 'https://tabs.ultimate-guitar.com/tab/creedence-clearwater-revival/fortunate-son-chords-15889',
 'https://tabs.ultimate-guitar.com/tab/janis-joplin/me-and-bobby-mcgee-chord

In [22]:
# df_pl = pd.DataFrame()
df_pl = []

for iLink in range(0, len(pl_tab_links)):
    song_info = scrape(pl_tab_links[iLink])
    dfSong = pd.concat([dfSong, pd.DataFrame.from_dict(song_info, orient='index').transpose()], axis=0)

dfSong = dfSong.reset_index(drop=True)
dfSong

,Song,Artist,Chords
0,island-in-the-sun,weezer,"Am,Em,D,C,G"
1,all-along-the-watchtower,jimi-hendrix,"Bb,Cm,Ab"
2,back-in-black,acdc,"A,D,C,B,E,G,A/C#"
3,rebel-rebel,david-bowie,"E,A,Bm,D"
4,should-i-stay-or-should-i-go,the-clash,"A,G,D,F"
5,sweet-emotion,aerosmith,"A,C,G,D"
6,another-one-bites-the-dust,queen,"Am,Em,A,C,B,G,F#m"
7,light-my-fire,the-doors,"Am7,D#,A#,A,D,Bm7,B,E,G#,C,F,G,F#m"
8,la-grange,zz-top,"A,D,C,F,Eb"
9,like-a-rolling-stone,bob-dylan,"Em,C,Dm7,F,Dm,G"


In [25]:
# # scrape links one by one for chords, save out a dictionary when successful
# for iSong in range(0, len(link_df)):
#     url = link_df['song_links'][iSong]
#     song_info = scrape(url, skill[iLevel])

#     if song_info['Chords']:
#         # set up name string
#         tmpName = [song_info['Artist'] + ' - ' + song_info['Song']]
#         fname = ' ' 
#         fname = fname.join(tmpName)

#         # open, write, save a csv under "Aritst - Song"
# #                 with open('ug_chords/'+fname+'.csv', 'w') as f:
#         with open('ug_chords10/'+fname+'.csv', 'w') as f:
#             for key in song_info.keys():
#                 f.write("%s,%s\n"%(key,song_info[key]))
                

dfSong.to_csv('playlist_df.csv', index=False)

In [4]:
dfSong = pd.read_csv('allSongsBasic.csv')
dfSong.head()


,Artist,Chords,Label,Song
0,weezer,"Am,Em,D,C,G",NaN,island-in-the-sun
1,jimi-hendrix,"Bb,Cm,Ab",NaN,all-along-the-watchtower
2,acdc,"A,D,C,B,E,G,A/C#",NaN,back-in-black
3,david-bowie,"E,A,Bm,D",NaN,rebel-rebel
4,the-clash,"A,G,D,F",NaN,should-i-stay-or-should-i-go


In [5]:
# put together a list of various standard major & minor chords
sl1 = ['A', 'Bb', 'B', 'C', 'C#', 'Db', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'Ab' ]
sl2 = [s + 'maj' for s in sl1]
sl3 = [s + 'm' for s in sl1]
sl4 = [s + 'min' for s in sl1]

standardList = sl1+sl2+sl3+sl4


In [30]:
# get feature dataframe for the known song.
nUniqueChords   = []
nHardChords     = []
chordDensity    = []
difficultyLabel = []

spotFeatures = {}

# spotFeatures['key'] = []
spotFeatures['tempo'] = []
# spotFeatures['duration_ms'] = []
spotFeatures['duration_m'] = []
spotFeatures['time_signature'] = []
spotFeatures['energy'] = []

for iSong in range(0, len(dfSong)):
    songName   = dfSong['Song'][iSong].replace('-', ' ').lower()
    artistName = dfSong['Artist'][iSong].replace('-', ' ').lower()

    # search spotify api
    result = sp.search(songName, limit=10) 
    nItems = len(result['tracks']['items'])

    # if our spotify search returns anything, extract desired vars
    if nItems>0:
        # match artist info, and get URI    
        foundMatch = [] # set up a counter...
        for iArtist in range(0, nItems):    
            if result['tracks']['items'][iArtist]['artists'][0]['name'].lower == artistName:
                songURI = result['tracks']['items'][iArtist]['uri']
                foundMatch.append(True)
            else:
                foundMatch.append(False) #... that keeps track of if we found a matching artist

        # if we didn't, let's assume the first artist was the correct one, and this was a 
        # problem with spelling/special chars/case sensitivity...
        if sum(foundMatch)==0:
            songURI = result['tracks']['items'][0]['uri']

        # pull audio features for a songURI
        features = sp.audio_features(songURI)

    #     spotFeatures['key'].append(features[0]['key'])
        spotFeatures['tempo'].append(features[0]['tempo'])
        spotFeatures['duration_m'].append((features[0]['duration_ms']/1000)/60)
        spotFeatures['time_signature'].append(features[0]['time_signature'])
        spotFeatures['energy'].append(features[0]['energy'])

    # if not, fill with NaNs
    else:
        spotFeatures['tempo'].append(float('nan'))
        spotFeatures['duration_m'].append(float('nan'))
        spotFeatures['time_signature'].append(float('nan'))
        spotFeatures['energy'].append(float('nan'))
        
    # get the chord information
    uniqChords = dfSong['Chords'][iSong].split(',')[1:]
    nUniqueChords.append(len(uniqChords))
    chordDensity.append(nUniqueChords[iSong]/spotFeatures['duration_m'][iSong])

    tmpHard = []
    for kCh in range(0, nUniqueChords[iSong]):
        tmpHard.append(uniqChords[kCh] not in standardList)

    nHardChords.append(sum(tmpHard))


In [31]:
spotFeatures = pd.DataFrame.from_dict(spotFeatures)
# spotFeatures.head()
allFeatures = spotFeatures

allFeatures['n_unique_chords']     = nUniqueChords
allFeatures['n_difficult_chords']  = nHardChords
allFeatures['chord_per_min']       = chordDensity

# allFeatures.drop(['key', 'duration_ms'], axis=1, inplace=True)
allFeatures


,tempo,duration_m,time_signature,energy,n_unique_chords,n_difficult_chords,chord_per_min
0,114.623,3.338450,4.0,0.8100,4,0,1.198161
1,113.253,4.013333,4.0,0.8050,2,0,0.498339
2,188.386,4.258217,4.0,0.7000,6,1,1.409041
3,125.988,4.579117,4.0,0.6860,3,0,0.655148
4,113.375,3.149783,4.0,0.8330,3,0,0.952446
5,99.437,4.568883,4.0,0.7600,3,0,0.656616
6,109.975,3.577550,4.0,0.5280,6,0,1.677125
7,124.618,7.162667,4.0,0.7250,12,3,1.675354
8,80.544,3.841333,4.0,0.6400,4,0,1.041305
9,95.261,6.160000,4.0,0.7210,5,1,0.811688


In [35]:
# import sqlite3

# conn = sqlite3.connect('/Users/aaronlevi/Documents/sql_db/chords_list.db')
# allFeatures.to_sql('features', con=conn, if_exists='append')

# cur.close()
# conn.close()


In [43]:
songName   = dfSong['Song'][8].replace('-', ' ').title()
artistName = dfSong['Artist'][8].replace('-', ' ').title()

# search spotify api
result = sp.search(songName, limit=10) 
nItems = len(result['tracks']['items'])

# match artist info, and get URI
for iArtist in range(0, nItems):    
    if result['tracks']['items'][iArtist]['artists'][0]['name'] == artistName:
       songURI = result['tracks']['items'][iArtist]['uri']

# if we successfully found the song in spotify, go ahead and gather featers
#     if songURI:
# pull audio features for a songURI
features = sp.audio_features(songURI)
print(artistName)

Zz Top


In [29]:
# # print(result['tracks']['items'][0]['artists'][0]['name'])
# print(artistName)
# print(songName)
# # print( result['tracks']['items'][0]['artists'][0]['name']==artistName )
# # print( result['tracks']['items'][0]['artists'][0]['name'].lower()==artistName.lower() ) 
# len(result['tracks']['items'])


In [88]:
import pickle
# load the model from disk
filename = 'dtree_model_0924.sav'
dtree = pickle.load(open(filename, 'rb'))

novel_song_label = dtree.predict(allFeatures)
novel_song_label

array([2, 1, 2, 0, 1, 2, 0, 2, 0, 0, 2, 2, 2, 0, 1, 2, 0, 0, 2, 0, 2, 1,
       2, 0, 0, 2, 1, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 1, 1, 1, 2, 2,
       0, 0, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2])

In [151]:
clist   = dfSong['Chords'][0].split(',')
nchords = len(clist)

nSameChords     = []
propSameChords  = []
exactSame       = []
similar_nchords = []
sim_nchords_ix  = []
for iSong in range(1, len(dfSong)):
    similar_nchords.append(len(dfSong['Chords'][iSong].split(',')) <= nchords )

#    if similar_nchords[iSong-1]==True & novel_song_label[iSong-1] == 0:
#     if similar_nchords[iSong-1]==True & novel_song_label[0] == novel_song_label[iSong-1]:
    if similar_nchords[iSong-1]==True:
        if float(novel_song_label[0]) == float(novel_song_label[iSong-1]):
            sim_nchords_ix.append(iSong)
            nSameChords.append( len([ele for ele in clist if(ele in dfSong['Chords'][iSong].split(','))]) )
            propSameChords.append( len([ele for ele in clist if(ele in dfSong['Chords'][iSong].split(','))]) / len(dfSong['Chords'][iSong].split(','))  )
#     nSameChords.append( len([ele for ele in clist if(ele in dfSong['Chords'][iSong].split(','))]) )
#     exactSame.append(nSameChords[iSong-1]==len(clist))

In [152]:
import numpy as np

ixMostSame = np.argmax(propSameChords)
ixMostSame = sim_nchords_ix[ixMostSame]

dfSong.iloc[ixMostSame]

Song      edge-of-seventeen
Artist         stevie-nicks
Chords               Em,D,C
Name: 23, dtype: object

In [145]:
dfSong.iloc[0]

Song      island-in-the-sun
Artist               weezer
Chords          Am,Em,D,C,G
Name: 0, dtype: object

In [146]:
print( [novel_song_label[0] , novel_song_label[ixMostSame]])

float(novel_song_label[0]) == float(novel_song_label[ixMostSame])

[2, 0]


False

In [101]:
'island-in-the-sun' in list(dfSong['Song'])

True

In [102]:
pl_tracks

['Whole Lotta Love - 1990 Remaster',
 'All Along the Watchtower',
 'Back In Black',
 'Paint It, Black - Mono',
 'Rebel Rebel - 1999 Remaster',
 'Come Together - Remastered 2009',
 'Should I Stay or Should I Go - Remastered',
 "Baba O'Riley",
 'Sweet Emotion',
 'Another One Bites The Dust - Remastered 2011',
 'Light My Fire',
 'La Grange - 2005 Remaster',
 'American Girl',
 'Paradise City',
 'House Of The Rising Sun',
 'Like a Rolling Stone',
 'Sunshine Of Your Love',
 'Sweet Home Alabama',
 'Fortunate Son',
 'Another Brick in the Wall, Pt. 2',
 'The Chain',
 'Me and Bobby McGee',
 'Hotel California - 2013 Remaster',
 'The Joker',
 'Born To Be Wild',
 "For What It's Worth",
 'More Than a Feeling',
 'Sultans of Swing',
 'Somebody to Love',
 'Walk On the Wild Side',
 'Rock And Roll All Nite',
 'Tom Sawyer',
 'Under Pressure - Remastered',
 'When Doves Cry',
 'Every Breath You Take - Remastered 2003',
 "(Don't Fear) The Reaper",
 'A Horse with No Name',
 "Rocket Man (I Think It's Going To 

In [110]:
print(dfSong.iloc[ixMostSame]['Song'].replace('-', ' ').title())
print(dfSong.iloc[ixMostSame]['Artist'].replace('-', ' ').title())
print(dfSong.iloc[ixMostSame]['Chords'])

Hurricane
Bob Dylan
Am,Em,C,F,Dm,G


In [109]:
dfSong.iloc[ixMostSame]

Song           hurricane
Artist         bob-dylan
Chords    Am,Em,C,F,Dm,G
Name: 28, dtype: object

In [23]:
pd.DataFrame.from_dict(spotFeatures)


,tempo,duration_m,time_signature,energy
0,113.139,3.739217,4,0.673
1,113.253,4.013333,4,0.805
2,113.253,4.013333,4,0.805
3,125.939,3.177783,4,0.483
4,113.375,3.149783,4,0.833
5,97.927,5.952217,4,0.919
6,115.991,5.982217,4,0.984
7,129.595,2.893333,4,0.631
8,129.595,2.893333,4,0.631
9,168.808,9.162667,4,0.828
